In [5]:
import configparser
import os
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date, timedelta
import os
import urllib
import requests
from urllib.error import *

In [2]:
config = configparser.ConfigParser()
config.read("capstone.cfg")

['capstone.cfg']

In [9]:
os.chdir(config["PATH"]["project"])
project_path = config["PATH"]["project"]

Create spark session. Add driver postgress to enable to load from existing postgres DB


In [4]:
# add driver postgress to enable to load from existing postgres DB
spark = SparkSession \
    .builder \
    .appName("US_weather") \
    .config("spark.jars", "postgresql-42.2.18.jar")\
    .config( "spark.driver.extraClassPath", "postgresql-42.2.18.jar")\
    .getOrCreate()

# Download weather data

In [15]:
weather_dir = "https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/"


In [17]:
weather_2020 = "2020.csv.gz"
url_weather = os.path.join(weather_dir,"by_year", weather_2020)
print(url_weather)

https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/2020.csv.gz


In [18]:
out_weather = os.path.join(project_path, "DATA", "WEATHER", weather_2020)
urllib.request.urlretrieve(url_weather, out_weather)

('/home/user/CODE/BIG_DATA/CAPSTONE_PROJECT/covid-analysis/DATA/WEATHER/2020.csv.gz',
 <http.client.HTTPMessage at 0x7ff5bfff1ad0>)

In [23]:
for weather_file in ["ghcnd-stations.txt", "ghcnd-states.txt", "ghcnd-countries.txt", "readme.txt"]:
    url = os.path.join(weather_dir, weather_file)
    out = os.path.join(project_path, "DATA", "WEATHER", weather_file)
    urllib.request.urlretrieve(url, out)

# Load weather records

In [98]:
weather = spark.read.csv(out_weather,  
                         schema = "station_id string, date string, measured string, v1 string, v2 string, v3 string, v4 string, v5 string")

In [99]:
weather.printSchema()

root
 |-- station_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- measured: string (nullable = true)
 |-- v1: string (nullable = true)
 |-- v2: string (nullable = true)
 |-- v3: string (nullable = true)
 |-- v4: string (nullable = true)
 |-- v5: string (nullable = true)



In [100]:
weather.show(10)

+-----------+--------+--------+---+----+----+---+----+
| station_id|    date|measured| v1|  v2|  v3| v4|  v5|
+-----------+--------+--------+---+----+----+---+----+
|AE000041196|20200101|    TMIN|168|null|null|  S|null|
|AE000041196|20200101|    PRCP|  0|   D|null|  S|null|
|AE000041196|20200101|    TAVG|211|   H|null|  S|null|
|AEM00041194|20200101|    PRCP|  0|null|null|  S|null|
|AEM00041194|20200101|    TAVG|217|   H|null|  S|null|
|AEM00041217|20200101|    TAVG|205|   H|null|  S|null|
|AEM00041218|20200101|    TMIN|148|null|null|  S|null|
|AEM00041218|20200101|    TAVG|199|   H|null|  S|null|
|AFM00040938|20200101|    PRCP| 23|null|null|  S|null|
|AFM00040938|20200101|    TAVG| 54|   H|null|  S|null|
+-----------+--------+--------+---+----+----+---+----+
only showing top 10 rows



In [101]:
weather.count()

34571064

# Load weather stations 

In [25]:
stations = spark.read.csv( os.path.join(project_path, "DATA", "WEATHER", "ghcnd-stations.txt"))

In [26]:
stations.printSchema()

root
 |-- _c0: string (nullable = true)



In [27]:
stations.show(10)

+--------------------+
|                 _c0|
+--------------------+
|ACW00011604  17.1...|
|ACW00011647  17.1...|
|AE000041196  25.3...|
|AEM00041194  25.2...|
|AEM00041217  24.4...|
|AEM00041218  24.2...|
|AF000040930  35.3...|
|AFM00040938  34.2...|
|AFM00040948  34.5...|
|AFM00040990  31.5...|
+--------------------+
only showing top 10 rows



In [65]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import MapType, StringType
from collections import OrderedDict

In [90]:
@udf(MapType( StringType(), StringType()))
def ParseStationsUDF(line):
    return{
        "station_id": line[0:11],
        "latitude" : line[13:20], 
        "longitude" : line[21:30], 
        "elevation" : line[31:38], 
        "state" : line[38:40], 
        "station_name" : line[41:]
        
    }

In [91]:
fields = OrderedDict( [
        ( "station_id" , "string"),
        ( "latitude" , "float"), 
        ("longitude" , "float"), 
        ("elevation" , "float"),
        ("state" , "string"), 
        ("station_name" , "string")
] )

#exprs = [ f"parsed['{field}'].cast({fld_type}) as {field}" for field, fld_type in fields.items() ]
exprs = [ f"CAST(parsed['{field}'] AS {fld_type}) AS {field}" for field, fld_type in fields.items() ]
exprs

["CAST(parsed['station_id'] AS string) AS station_id",
 "CAST(parsed['latitude'] AS float) AS latitude",
 "CAST(parsed['longitude'] AS float) AS longitude",
 "CAST(parsed['elevation'] AS float) AS elevation",
 "CAST(parsed['state'] AS string) AS state",
 "CAST(parsed['station_name'] AS string) AS station_name"]

In [92]:
stations_parsed = stations.withColumn("parsed", ParseStationsUDF("_c0")).selectExpr( *exprs)

In [93]:
stations_parsed.printSchema()

root
 |-- station_id: string (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- elevation: float (nullable = true)
 |-- state: string (nullable = true)
 |-- station_name: string (nullable = true)



In [94]:
stations_parsed.take(10)

[Row(station_id='ACW00011604', latitude=17.11669921875, longitude=-61.78329849243164, elevation=10.100000381469727, state='  ', station_name='ST JOHNS COOLIDGE FLD                       '),
 Row(station_id='ACW00011647', latitude=17.13330078125, longitude=-61.78329849243164, elevation=19.200000762939453, state='  ', station_name='ST JOHNS                                    '),
 Row(station_id='AE000041196', latitude=25.33300018310547, longitude=55.516998291015625, elevation=34.0, state='  ', station_name='SHARJAH INTER. AIRP            GSN     41196'),
 Row(station_id='AEM00041194', latitude=25.2549991607666, longitude=55.36399841308594, elevation=10.399999618530273, state='  ', station_name='DUBAI INTL                             41194'),
 Row(station_id='AEM00041217', latitude=24.433000564575195, longitude=54.6510009765625, elevation=26.799999237060547, state='  ', station_name='ABU DHABI INTL                         41217'),
 Row(station_id='AEM00041218', latitude=24.261999130249023

In [81]:
stations_parsed.select("STATE").distinct().count()

76

In [95]:
# states are US and Canadian states
stations_parsed.select("STATE").distinct().show()

+-----+
|STATE|
+-----+
|   AZ|
|   SC|
|   NS|
|   NL|
|   LA|
|   MN|
|   NJ|
|   DC|
|   OR|
|   UM|
|   NT|
|   VA|
|   QC|
|   RI|
|   KY|
|   WY|
|   BC|
|   NH|
|   MI|
|     |
+-----+
only showing top 20 rows



In [96]:
stations_parsed.where( col("STATE") == "BC").show()

+-----------+--------+---------+---------+-----+--------------------+
| station_id|latitude|longitude|elevation|state|        station_name|
+-----------+--------+---------+---------+-----+--------------------+
|CA001010066| 48.8667|-123.2833|      4.0|   BC|ACTIVE PASS      ...|
|CA001010235|    48.4|-123.4833|     17.0|   BC|ALBERT HEAD      ...|
|CA001010595| 48.5833|-123.5167|     85.0|   BC|BAMBERTON OCEAN C...|
|CA001010720|    48.5|   -124.0|    351.0|   BC|BEAR CREEK       ...|
|CA001010774|    48.5|  -123.35|     61.0|   BC|BEAVER LAKE      ...|
|CA001010780| 48.3333|-123.6333|     12.0|   BC|BECHER BAY       ...|
|CA001010960|    48.6|-123.4667|     38.0|   BC|BRENTWOOD BAY 2  ...|
|CA001010961| 48.5667|  -123.45|     31.0|   BC|BRENTWOOD CLARKE ...|
|CA001010965| 48.5667|-123.4333|     91.0|   BC|BRENTWOOD W SAANI...|
|CA001011467| 48.5833|-123.4167|     53.0|   BC|CENTRAL SAANICH V...|
|CA0010114F6| 48.5667|   -123.4|     38.0|   BC|CENTRAL SAANICH I...|
|CA0010114FF|   48.5

In [97]:
stations_parsed.count()

118492